<font color="#CC3D3D"><p>
# [Competition] Building a `LightGBM` Model with `Pipeline+Optuna`

In [1]:
LGBM_VERSION = 4.0   # submission 화일명에 사용

In [2]:
%matplotlib inline
import warnings
warnings.filterwarnings("ignore")

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import FunctionTransformer
from sklearn.base import BaseEstimator, TransformerMixin
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import PowerTransformer 
from sklearn.preprocessing import OrdinalEncoder
from sklearn.decomposition import PCA
from sklearn.feature_selection import SelectPercentile
from sklearn.model_selection import train_test_split, cross_val_score, cross_validate, ShuffleSplit
from sklearn.metrics import mean_squared_error
from sklearn import set_config
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.compose import TransformedTargetRegressor
from lightgbm import LGBMRegressor
import lightgbm as lgb
from category_encoders import CatBoostEncoder
import optuna
from optuna.distributions import CategoricalDistribution, IntDistribution, FloatDistribution
from optuna.integration import OptunaSearchCV
from optuna.integration.lightgbm import LightGBMTunerCV
from scipy.sparse import csr_matrix
import re

In [3]:
X_train = pd.read_csv('X_train.csv', encoding='cp949').drop(columns='ID')
y_train = pd.read_csv('y_train.csv', encoding='cp949').Salary

X_test = pd.read_csv('X_test.csv', encoding='cp949')
test_id = X_test.ID
X_test = X_test.drop(columns='ID')

In [4]:
## 어학시험 결측치 처리
X_train['어학시험'] = X_train['어학시험'].fillna('없음')
X_train = X_train.replace({'어학시험' :' '},'없음') 

X_test['어학시험'] = X_test['어학시험'].fillna('없음')
X_test = X_test.replace({'어학시험' :' '},'없음') 

In [5]:
df=pd.concat([X_train, X_test])

In [6]:
for i , j in  zip(df['대학전공'].str.contains('서양'), df['대학전공']) :
    if i ==True:
        df['대학전공'].replace(j,'서양화',inplace=True)
        
for i , j in  zip(df['대학전공'].str.contains('컴퓨터정보'), df['대학전공']) :
    if i ==True:
        df['대학전공'].replace(j,'컴퓨터정보공학',inplace=True)
        
for i , j in  zip(df['대학전공'].str.contains('섬유예술'), df['대학전공']) :
    if i ==True:
        df['대학전공'].replace(j,'섬유예술학',inplace=True)
        
for i , j in  zip(df['대학전공'].str.contains('영어교육'), df['대학전공']) :
    if i ==True:
        df['대학전공'].replace(j,'영어교육학',inplace=True)
        
for i , j in  zip(df['대학전공'].str.contains('보험'), df['대학전공']) :
    if i ==True:
        df['대학전공'].replace(j,'금융보험학',inplace=True)

for i , j in  zip(df['대학전공'].str.contains('세무'), df['대학전공']) :
    if i ==True:
        df['대학전공'].replace(j,'세무회계학',inplace=True)
        
for i , j in  zip(df['대학전공'].str.contains('천연섬유'), df['대학전공']) :
    if i ==True:
        df['대학전공'].replace(j,'천연섬유학',inplace=True)

for i , j in  zip(df['대학전공'].str.contains('섬유디자인'), df['대학전공']) :
    if i ==True:
        df['대학전공'].replace(j,'섬유디자인학',inplace=True)
        
for i , j in  zip(df['대학전공'].str.contains('간호'), df['대학전공']) :
    if i ==True:
        df['대학전공'].replace(j,'간호학',inplace=True)
        
for i , j in  zip(df['대학전공'].str.contains('전기전자제어'), df['대학전공']) :
    if i ==True:
        df['대학전공'].replace(j,'전기전자제어학',inplace=True)

for i , j in  zip(df['대학전공'].str.contains('전기전자정보'), df['대학전공']) :
    if i ==True:
        df['대학전공'].replace(j,'전기전자정보',inplace=True)
        
for i , j in  zip(df['대학전공'].str.contains('전자통신'), df['대학전공']) :
    if i ==True:
        df['대학전공'].replace(j,'전자통신학',inplace=True)

for i , j in  zip(df['대학전공'].str.contains('전자정보통신'), df['대학전공']) :
    if i ==True:
        df['대학전공'].replace(j,'전자통신학',inplace=True)        

In [7]:
df['대학전공']=df['대학전공'].replace(
{
    '경영':'경영학',
    '경영학':'경영학',
    '경영학 전공':'경영학',
    '경영학과':'경영학',
    '경영학과 (주간)':'경영학',
    '경영학과(주간)':'경영학',
    '경영학부':'경영학',
    '경영학부 경영학':'경영학',
    '경영학부 경영학 전공':'경영학',
    '경영학사':'경영학',
    '경영정보':'경영정보학',
    '경영정보학과':'경영정보학',
    'MIS(경영정보시스템)':'경영정보학',
    '인터넷경영정보':'경영정보학',
    '경영정보(MIS)':'경영정보학',
    '경영정보과':'경영정보학',
    '경영정보시스템':'경영정보학',
    '경영정보과학부':'경영정보학',
    'MIS':'경영정보학',
    '경영(MIS)':'경영정보학',
    '정보처리(MIS)':'경영정보학',
    '경제학과':'경제학',
    '경제':'경제학',
    '경제학부':'경제학',
    '경제과':'경제학',
    '경제학사':'경제학',
    '컴퓨터공학과':'컴퓨터공학',
    '컴퓨터':'컴퓨터공학',
    '컴퓨터학과':'컴퓨터공학',
    '컴퓨터과학':'컴퓨터공학',
    '컴퓨터학':'컴퓨터공학',
    '컴퓨터과학과':'컴퓨터공학',
    '컴퓨터 공학':'컴퓨터공학',
    '컴퓨터공':'컴퓨터공학',
    '컴퓨터 공학과':'컴퓨터공학',
    '컴퓨터 과학과':'컴퓨터공학',
    '컴퓨터과':'컴퓨터공학',
    '컴퓨터공학전공':'컴퓨터공학',
    '통신컴퓨터공학부 컴퓨터전공':'컴퓨터공학',
    '컴퓨터 학부':'컴퓨터공학',
    '통신`컴퓨터공학분 컴퓨터전공':'컴퓨터공학',
    '전자공학과':'전자공학',
    '전자공학부':'전자공학',
    '기계전자공학부 전자 전공':'전자공학',
    '전기전자공학과':'전기전자공학',
    '전기전자공학부':'전기전자공학',
    '공대 전기전자공학과':'전기전자공학',
    '전기전자':'전기전자공학',
    '전자전기':'전기전자공학',
    '전기전자공':'전기전자공학',
    '전자전기공':'전기전자공학',
    '전자전기공':'전기전자공학',
    '전기전자 공학부':'전기전자공학',
    '전기,전자':'전기전자공학',
    '전자전기공학':'전기전자공학',
    '전기공학과':'전기공학',
    '전기':'전기공학',
    '전기공':'전기공학',
    '전기과':'전기공학',
    '전기 공학과':'전기공학',
    '전기공학부':'전기공학',
    '전기공학전공':'전기공학',
    '전기 공학':'전기공학',
    '전기학과':'전기공학',
    '전기공항':'전기공학',
    '전기공화과':'전기공학',
    '전기(전자)':'전기공학',
    '정보통신':'정보통신학',
    '정보통신공학과':'정보통신학',
    '정보통신공':'정보통신학',
    '정보통신학과':'정보통신학',
    '정보통신공학부':'정보통신학',
    '정보통신학':'정보통신학',
    '정보통신과':'정보통신학',
    '정보통신 공학과':'정보통신학',
    '정보통신학과(구 전자공학과)':'정보통신학',
    '정보통신전공':'정보통신학',
    '전기전자공학부 정보통신전공':'정보통신학',
    '정보통신 공학':'정보통신학',
    '정보통신공학사':'정보통신학',
    '컴퓨터정보공학부 정보통신공학전공':'정보통신학',
    '정보통신공학전공':'정보통신학',
    '영어영문':'영어영문학',
    '영어영문학과':'영어영문학',
    '영어학과':'영어영문학',
    '영어학':'영어영문학',
    '영어과':'영어영문학',
    '영어영문과':'영어영문학',
    '영어 영문학':'영어영문학',
    '영어 영문학과':'영어영문학',
    '영어 영문':'영어영문학',
    '영어영문학부':'영어영문학',
    '영어 영문과':'영어영문학',
    '영어영문학(야)':'영어영문학',
    '영어영문학사':'영어영문학',
    '영문영어':'영어영문학',
    '영어 전공':'영어영문학',
    '기계공학과':'기계공학',
    '기계공학부':'기계공학',
    '기계':'기계공학',
    '기계공':'기계공학',
    '기계과':'기계공학',
    '기계 공학과':'기계공학',
    '기계공악':'기계공학',
    '기계 공학':'기계공학',
    '기계공학 전공':'기계공학',
    '기계고학부':'기계공학',
    '기계공확과':'기계공학',
    '산업공학과':'산업공학',
    '산업공학전공':'산업공학',
    '회계학과':'회계학',
    '회계':'회계학',
    '회계헉':'회계학',
    '회계학과(야)':'회계학',
    '상경학부(회계전공)':'회계학',
    '경영학부(회계학)':'회계학',
    '회계확과':'회계학',
    '회계학과졸업':'회계학',
    '회계학':'회계학',
    '경영경제':'경영경제학',
    '경영경제 (주)':'경영경제학',
    '경영경제 전공':'경영경제학',
    '경영경제학과':'경영경제학',
    '행정학과':'행정학',
    '행정':'행정학',
    '행정학부':'행정학',
    '행정과':'행정학',
    '행정학전공':'행정학',
    '행정학과 졸업':'행정학',
    '법정대 행정학':'행정학',
    '법과대학 행정학':'행정학',
    '법학과':'법학',
    '법':'법학',
    '법학전공':'법학',
    '법(야)':'법학',
    '법정학부':'법학',
    '법과대학':'법학',
    '법학학사':'법학',
    '무역학과':'무역학',
    '무역':'무역학',
    '국제무역학과':'무역학',
    '무역학전공':'무역학',
    '경영학부 무역학':'무역학',
    '국제무역':'무역학',
    '무역과':'무역학',
    '경상학부 무역학':'무역학',
    '경상 무역학과':'무역학',
    '경상학부 무역학과':'무역학',
    '무역학과 졸업':'무역학',
    '무역전공':'무역학',
    '무역학부':'무역학',
    '화학과':'화학',
    '화학학과':'화학',
    '화학 전공':'화학',
    '공과대학 화학공학과':'화학공학',
    '화학 공학':'화학공학',
    '화학 공학과':'화학공학',
    '화학공':'화학공학',
    '화학공학과':'화학공학',
    '화학공항과':'화학공학',
    '화학공확과':'화학공학',
    '디자인학부 산업디자인과':'산업디자인학',
    '디자인학구 산업지자인전공':'산업디자인학',
    '산업 디자인':'산업디자인학',
    '산업디자인':'산업디자인학',
    '산업디자인 학과':'산업디자인학',
    '산업디자인과':'산업디자인학',
    '산업디자인부':'산업디자인학',
    '산업디자인하과':'산업디자인학',
    '산업디자인학과':'산업디자인학',
    '산업디자인학부':'산업디자인학',
    '의류학과':'의류학',
    '의류':'의류학',
    '의류헉과':'의류학',
    '디자인학 시각디자인전공':'시각디자인학',
    '비주얼 디자인':'시각디자인학',
    '비쥬얼디자인(시각)':'시각디자인학',
    '비쥬얼디자인(시각디자인)':'시각디자인학',
    '산업디자인 시각미디어전공':'시각디자인학',
    '산업디자인(시각)':'시각디자인학',
    '산업디자인(시각디자인 전공)':'시각디자인학',
    '산업디자인(시각디자인)':'시각디자인학',
    '산업디자인과 시각디자인전공':'시각디자인학',
    '산업디자인과시각디자인전공':'시각디자인학',
    '산업디자인학과 시각디자인전공':'시각디자인학',
    '산업디자인학과(시각디자인전공)':'시각디자인학',
    '산업미술학과 시각디자인':'시각디자인학',
    '시각 디자인':'시각디자인학',
    '시각디자인':'시각디자인학',
    '시각디자인 전공':'시각디자인학',
    '시각디자인과':'시각디자인학',
    '시각디자인학과':'시각디자인학',
    '중어중문':'중어중문학',
    '중어중문학과':'중어중문학',
    '중어중국학과':'중어중문학',
    '중어중문과':'중어중문학',
    '중어학과':'중어중문학',
    '중어중국':'중어중문학',
    '중어중국학':'중어중문학',
    '중어.중문학과':'중어중문학',
    '중어 중국학과':'중어중문학',
    '중국어학과':'중어중문학',
    '중국어과':'중어중문학',
    '중국어학':'중어중문학',
    '중국어중국학':'중어중문학',
    '중국어중국문학':'중어중문학',
    '중국어전공':'중어중문학',
    '중국어':'중어중문학',
    '중국어 학과':'중어중문학',
    '전자계산학과':'전자계산학',
    '전자계산':'전자계산학',
    '전자계산과':'전자계산학',
    '전자계산공학':'전자계산학',
    '전자계산학과(야)':'전자계산학',
    '전자계산공학과':'전자계산학',
    '컴퓨터공학(전자계산학)':'전자계산학',
    '일어일문':'일어일문학',
    '일어일문학과':'일어일문학',
    '일어일본과':'일어일문학',
    '일어일본학과':'일어일문학',
    '일어일문과':'일어일문학',
    '일어일본':'일어일문학',
    '일어':'일어일문학',
    '일어 일본학과':'일어일문학',
    '일어일문학전공':'일어일문학',
    '일어과':'일어일문학',
    '일어일문확과':'일어일문학',
    '일어일분학과':'일어일문학',
    '일어일본어':'일어일문학',
    '일어 일문학':'일어일문학',
    '관광 경영':'관광경영학',
    '관광경여':'관광경영학',
    '관광경영':'관광경영학',
    '관광경영학':'관광경영학',
    '관광경영학과':'관광경영학',
    '관광경영학과 경영학사':'관광경영학',
    '관광경영학전공':'관광경영학',
    '건축공학과':'건축공학',
    '건축공':'건축공학',
    '건축 공학과':'건축공학',
    '건축공학부':'건축공학',
    '패션디자인':'패션디자인학',
    '패션디자인학과':'패션디자인학',
    '패션디자인과':'패션디자인학',
    '패션디자인전공':'패션디자인학',
    '패션 디자인과':'패션디자인학',
    '의상섬유학부 패션디자인 전공':'패션디자인학',
    '패션디자인학부':'패션디자인학',
    '패션디자인 전공':'패션디자인학',
    '미술대학 패션디자인과':'패션디자인학',
    '패션디자인전공':'패션디자인학',
    '패션 디자인학과':'패션디자인학',
    '디자인학부 패션디자인과':'패션디자인학',
    '디자인대학 패션디자인학과':'패션디자인학',
    '미술대학 패션디자인학부':'패션디자인학',
    '패션지자인':'패션디자인학',
    '의상섬유학부 패션디자인':'패션디자인학',
    '패션디자인 2000.03~2008.02':'패션디자인학',
    '수학과':'수학',
    '수학학과':'수학',
    '수학전공':'수학',
    '전산통계':'전산통계학',
    '전산통계학과':'전산통계학',
    '전산통계학':'전산통계학',
    '전산 통계학':'전산통계학',
    '전산통게':'전산통계학',
    '전산통계학교':'전산통계학',
    '데이터정보(구 전산통계)':'전산통계학',
    '국어국문':'국어국문학',
    '국어국문학과':'국어국문학',
    '한국어문학':'국어국문학',
    '국어국문과':'국어국문학',
    '국어국문학가':'국어국문학',
    '인문학부 국어국문학과':'국어국문학',
    '국어국문학교':'국어국문학',
    '기계설계':'기계설계공학',
    '기계설계학과':'기계설계공학',
    '기계설계공학과':'기계설계공학',
    '기계설계학':'기계설계공학',
    '기게설계과':'기계설계공학',
    '기계공학부(기계설계)':'기계설계공학',
    '기계공학(기계설계)':'기계설계공학',
    '기계 설계':'기계설계공학',
    '기계설계 공학':'기계설계공학',
    '기계/설계':'기계설계공학',
    '기계설게공학과':'기계설계공학',
    '기계설계 공 학과':'기계설계공학',
    '신문방송':'신문방송학',
    '신문방송학과':'신문방송학',
    '신문방속학':'신문방송학',
    '언론정보학과(신문방송학과)':'신문방송학',
    '언론정보학과前신문방송학과':'신문방송학',
    '응용통계':'응용통계학',
    '응용통계학과':'응용통계학',
    '응용통계(경제학사)':'응용통계학',
    '상경 응용통계':'응용통계학',
    '상경대학 응용통계':'응용통계학',
    '응용통계학과(야간)':'응용통계학',
    '응용통계(상경)':'응용통계학',
    '경상대 응용통계':'응용통계학',
    '의상디자인':'의상디자인학',
    '의상디자인학과':'의상디자인학',
    '의상디자인과':'의상디자인학',
    '의생활학부 의상디자인':'의상디자인학',
    '의상 디자인':'의상디자인학',
    '의상 디자인과':'의상디자인학',
    '의상디자인전공':'의상디자인학',
    '의상 디자인학과':'의상디자인학',
    '공연예술학과 의상디자인':'의상디자인학',
    '의상디자인전공(대졸4)':'의상디자인학',
    '의상디자인학과졸':'의상디자인학',
    '사회학과':'사회학',
    '사회':'사회학',
    '토목공학과':'토목공학',
    '토목':'토목공학',
    '토목공':'토목공학',
    '토목공학부':'토목공학',
    '토목과':'토목공학',
    '토목학과':'토목공학',
    '토목공하과':'토목공학',
    '토목 공학과':'토목공학',
    '불어불문':'불어불문학',
    '불어불문학과':'불어불문학',
    '불어불문과':'불어불문학',
    '사학과':'사학',
    '역사학과':'사학',
    '역사학':'사학',
    '역사학전공':'사학',
    '인문학부 역사학과':'사학',
    '철학과':'철학',
    '철학전공':'철학',
    '의상학과':'의상학',
    '의상':'의상학',
    '의상학부':'의상학',
    '의상전공':'의상학',
    '물리학과':'물리학',
    '물리':'물리학',
    '물리&#54673':'물리학',
    '물리과':'물리학',
    '과학기술학부 물리학과':'물리학',
    '물리락과':'물리학',
    '일본어':'일어일문학',
    '일본어학과':'일어일문학',
    '일본어과':'일어일문학',
    '일본어학':'일어일문학',
    '일본어 학과':'일어일문학',
    '일본어 문학':'일어일문학',
    '일본어 전공':'일어일문학',
    '환경공학과':'환경공학',
    '환경학':'환경공학',
    '환경과학':'환경공학',
    '환경':'환경공학',
    '환경공':'환경공학',
    '환경학과':'환경공학',
    '환경과':'환경공학',
    '정치외교':'정치외교학',
    '정치외교학과':'정치외교학',
    '정치 외교 학과':'정치외교학',
    '정치 외교학과':'정치외교학',
    '독어독문':'독어독문학',
    '독어독문학과':'독어독문학',
    '독어학':'독어독문학',
    '독어 독문':'독어독문학',
    '독어독문 학과':'독어독문학',
    '독어독문과':'독어독문학',
    '생물학과':'생물학',
    '생물':'생물학',
    '생물과':'생물학',
    '사회복지':'사회복지학',
    '사회복지학과':'사회복지학',
    '사회복지과':'사회복지학',
    '시회복지학과':'사회복지학',
    '사회복지학부':'사회복지학',
    '금속공학과':'금속공학',
    '금속공':'금속공학',
    '금속':'금속공학',
    '금속공학교':'금속공학',
    '금속 공학과':'금속공학',
    '응용소재공학(구 금속공학)':'금속공학',
    '광고홍보':'광고홍보학',
    '광고홍보학과':'광고홍보학',
    '광고학과':'광고홍보학',
    '언론정보 광고홍보':'광고홍보학',
    '광고/홍보':'광고홍보학',
    '광고&PR':'광고홍보학',
    '광고홍':'광고홍보학',
    '광고,홍보':'광고홍보학',
    '경영학부 광고홍보':'광고홍보학',
    '광고학':'광고홍보학',
    '언론정보학부 광고홍보학 전공':'광고홍보학',
    '경영학부 광고.홍보학과':'광고홍보학',
    '광고':'광고홍보학',
    '체육학과':'체육학',
    '체육':'체육학',
    '체육과':'체육학',
    '체육대학':'체육학',
    '체육하과':'체육학',
    '체육학사':'체육학',
    '체육학전공 4학년':'체육학',
    '건강보건대학/체육학과':'체육학',
    '문헌정보':'문헌정보학',
    '문헌정보학과':'문헌정보학',
    '교육':'교육학',
    '전산':'전산학',
    '전산학과':'전산학',
    '전산과':'전산학',
    '전산학부':'전산학',
    '의류직물':'의류직물학',
    '의류직물학과':'의류직물학',
    '의류 직물학과':'의류직물학',
    '의류직물학전공':'의류직물학',
    '의류 직물':'의류직물학',
    '호텔관광경영':'호텔관광경영학',
    '호텔관광경영학과':'호텔관광경영학',
    '호텔관광 경영학과':'호텔관광경영학',
    '호텔관광경영하과':'호텔관광경영학',
    '호텔 관광 경영학과':'호텔관광경영학',
    '호텔.관광경영학':'호텔관광경영학',
    '호텔관광경영과':'호텔관광경영학',
    '호텔관관경영학과':'호텔관광경영학',
    '호텔관광':'호텔관광경영학',
    '호텔 관광경영학과':'호텔관광경영학',
    '호텔.관광 경영학과':'호텔관광경영학',
    '호텔관광경영대학':'호텔관광경영학',
    '식품영양':'식품영양학',
    '식품영양학과':'식품영양학',
    '심리학과':'심리학',
    '심리':'심리학',
    '식품공학과':'식품공학',
    '식품공':'식품공학',
    '식품':'식품공학',
    '재료공학과':'재료공학',
    '재료공':'재료공학',
    '재료':'재료공학',
    '재료 공학':'재료공학',
    '재료공합':'재료공학',
    '영어':'영어영문학',
    '영문학':'영어영문학',
    '영문학과':'영어영문학',
    '영문':'영어영문학',
    '영문과':'영어영문학',
    '영문학 전공':'영어영문학',
    '영여영문':'영어영문학',
    '영여영문학과':'영어영문학',
    '영여영문학':'영어영문학',
    '경영회계':'경영회계학',
    '경영회계학과':'경영회계학',
    '경영회계학부':'경영회계학',
    '경영/회계':'경영회계학',
    '경영.회계':'경영회계학',
    '경영.회계학과':'경영회계학',
    '경영,회계학':'경영회계학',
    '경영/회계학과':'경영회계학',
    '경영,회계학과':'경영회계학',
    '경영 회계':'경영회계학',
    '국제통상':'국제통상학',
    '국제통상학과':'국제통상학',
    '국제 통상':'국제통상학',
    '국제통상 학과':'국제통상학',
    '전자공':'전자공학',
    '통계학과':'통계학',
    '통계':'통계학',
    '경제학부 통계학':'통계학',
    '경제학부 통계학 전공':'통계학',
    '경제학부 통계학과':'통계학',
    '경제학부 통계학 전공':'통계학',
    '생명과학과':'생명과학',
    '생명과학부':'생명과학',
    '생명과학 전공':'생명과학',
    '생명과학학과':'생명과학',
    '생명과학전공':'생명과학',
    '도시공학과':'도시공학',
    '도시공':'도시공학',
    '도시공학전공':'도시공학',
    '제어계측':'제어계측학',
    '제어계측공학과':'제어계측학',
    '제어계측공':'제어계측학',
    '제어 계측':'제어계측학',
    '제어계측공학전공':'제어계측학',
    '제어계측공학 전공':'제어계측학',
    '사회체육':'사회체육학',
    '사회체육학과':'사회체육학',
    '사회체육과':'사회체육학',
    '사회체육 학과':'사회체육학',
    '호텔경영':'호텔경영학',
    '호텔경영학과':'호텔경영학',
    '호텔경영학':'호텔경영학',
    '호텔관광경영학부 호텔경영전공':'호텔경영학',
    '호텔경영전공':'호텔경영학',
    '경제무역':'경제무역학',
    '경제무역학과':'경제무역학',
    '경제/무역학':'경제무역학',
    '경제,무역':'경제무역학',
    '경제무역과':'경제무역학',
    '경제. 무역':'경제무역학',
    '경제.무역':'경제무역학',
    '가정관리':'가정관리학',
    '가정관리학과':'가정관리학',
    '가족환경(가정관리)':'가정관리학',
    '가정관리(현재아동가족학과로변경됨)':'가정관리학',
    '중국학과':'중국학',
    '중국':'중국학',
    '국제학부(중국학)':'중국학',
    '전자재료공학과':'전자재료공학',
    '전자재료':'전자재료공학',
    '전자재료공':'전자재료공학',
    '전자재료(전기전자공학부)':'전자재료공학',
    '전자재료 공학':'전자재료공학',
    '신학과':'신학',
    '신학부':'신학',
    '섬유공학과':'섬유공학',
    '의상섬유학부 섬유공학과':'섬유공학',
    '섬유 공학과':'섬유공학',
    '산업공':'산업공학',
    '유전공학과':'유전공학',
    '유전공':'유전공학',
    '생명과학 유전공':'유전공학',
    '멀티미디어':'멀티미디어학',
    '멀티미디어학과':'멀티미디어학',
    '멀티미디어학부':'멀티미디어학',
    '멀티미디어과':'멀티미디어학',
    '전산정보':'전산정보학',
    '정보전산':'전산정보학',
    '전산정보학과':'전산정보학',
    '정보전산학부':'전산정보학',
    '정보전산학':'전산정보학',
    '패션산업학과':'패션산업학',
    '의류패션산업':'패션산업학',
    '패션산업':'패션산업학',
    '패션산업학부':'패션산업학',
    '의류패션산업전공':'패션산업학',
    '의류패션산업학과':'패션산업학',
    '패션산업학':'패션산업학',
    '생활과학부 패션산업전공':'패션산업학',
    '패션산업전공':'패션산업학',
    '정보시스템':'정보시스템공학',
    '정보시스템학':'정보시스템공학',
    '정보시스템공학과':'정보시스템공학',
    '정보시스템 공학':'정보시스템공학',
    '건축':'건축학',
    '건축학과':'건축학',
    '건축과':'건축학',
    '신소재공학과':'신소재공학',
    '신소재':'신소재공학',
    '신소재학과':'신소재공학',
    '신소재공학':'신소재공학',
    '신소재 공학':'신소재공학',
    '신소재 공학과':'신소재공학',
    '신소재(고분재)공학과':'신소재공학',
    '신소재공학과(재료)':'신소재공학',
    '금속재료공학부 신소재공학':'신소재공학',
    '고분자공(신소재공학)':'신소재공학',
    '신소재공학부':'신소재공학',
    '신소재(고분자)공학':'신소재공학',
    '신소재과':'신소재공학',
    '자원공학과':'자원공학',
    '자원공':'자원공학',
    '공예학과':'공예학',
    '공예과':'공예학',
    '공예':'공예학',
    '가정학과':'가정학',
    '가정':'가정학',
    '가정학(인테리어)':'가정학',
    '가정학부':'가정학',
    '산업 경영학과':'산업경영공학',
    '산업경영':'산업경영공학',
    '산업경영공학과':'산업경영공학',
    '산업경영공학부':'산업경영공학',
    '산업경영학과':'산업경영공학',
    '전자전기공학부':'전기전자공학',
    '전자공학전공':'전자공학',
    '전자 공학':'전자공학',
    '전자공하과':'전자공학',
    '전자':'전자공학',
    '전자과':'전자공학',
    '전자전공':'전자공학',
    '전자 공학과':'전자공학'
})

In [8]:
df['근무형태'] = df['근무형태'].fillna('신입')

df['근무형태']=df['근무형태'].replace(
{
'정규직, 계약직, ':'정규직, 계약직',
'정규직, ':'정규직',
'정규직, 계약직, 해외취업, ':'정규직, 계약직, 해외취업',
'정규직, 계약직, 파견직, ':'정규직, 계약직, 파견직',
'정규직, 계약직, 해외취업, 파견직, ':'정규직, 계약직, 해외취업, 파견직',
'계약직, ':'계약직',
'정규직, 계약직, 해외취업, 인턴, ':'정규직, 계약직, 해외취업, 인턴',
'정규직, 해외취업, ':'정규직, 해외취업',
'정규직, 계약직, 인턴, 파견직, ':'정규직, 계약직, 인턴, 파견직',
'정규직, 계약직, 인턴, ':'정규직, 계약직, 인턴',
'정규직, 파견직, ':'정규직, 파견직',
'인턴, ':'인턴',
'정규직, 계약직, 해외취업, 인턴, 파견직, ':'정규직, 계약직, 해외취업, 인턴, 파견직',
'정규직, 해외취업, 파견직, ':'정규직, 해외취업, 파견직',
'계약직, 파견직, ':'계약직, 파견직',
'계약직, 인턴, ':'계약직, 인턴',
'정규직, 인턴, ':'정규직, 인턴',
'계약직, 해외취업, 파견직, ':'계약직, 해외취업, 파견직',
'정규직, 해외취업, 인턴, ':'정규직, 해외취업, 인턴',
'정규직, 계약직, 해외취업, 병역특례, 인턴, 파견직, ':'정규직, 계약직, 해외취업, 병역특례, 인턴, 파견직',
})

In [9]:
df['근무경력'].str.split(" ").str[0]
df['근무경력'].str.split(" ").str[-1]

df['연']=df['근무경력'].str.split(" ").str[0]
df['개월']=df['근무경력'].str.split(" ").str[-1]
df1=pd.concat([df.연,df.개월],axis=1)

df1['연'].loc[df1['연'] == '0개월'] = '0년'
df1['연'].loc[df1['연'] == '1개월'] = '0년'
df1['연'].loc[df1['연'] == '2개월'] = '0년'
df1['연'].loc[df1['연'] == '3개월'] = '0년'
df1['연'].loc[df1['연'] == '4개월'] = '0년'
df1['연'].loc[df1['연'] == '5개월'] = '0년'
df1['연'].loc[df1['연'] == '6개월'] = '0년'
df1['연'].loc[df1['연'] == '7개월'] = '0년'
df1['연'].loc[df1['연'] == '8개월'] = '0년'
df1['연'].loc[df1['연'] == '9개월'] = '0년'
df1['연'].loc[df1['연'] == '10개월'] = '0년'
df1['연'].loc[df1['연'] == '11개월'] = '0년'

df1['A'] = df1['연'].str.rstrip("년")
df1['B'] = df1['개월'].str.rstrip("개월")
df1['A'].unique()

df1['연차를 개월로 환산'] = df1['A'].astype(dtype='int64')*12
df1['B'] = df1['B'].astype(dtype='int64')*1
df1['개월수'] = df1['연차를 개월로 환산']+df1['B']
df['근무경력'] = df1['개월수']
df['근무경력'].value_counts()

df=df.drop(['연'],axis=1)
df=df.drop(['개월'],axis=1)

In [10]:
df['대학성적'] = df['대학성적'].fillna(df['대학성적'].mean())

In [11]:
df['어학시험']=df['어학시험'].replace(
{                               
    'TOEIC':'영어',
    'TOEFL':'영어',
    'OPIc':'영어',
    'OPI':'영어',
    'TEPs':'영어',
    'TEPS':'영어',
    'TOEIC Speaking':'영어',
    'IELTS':'영어',
    'G-TELP':'영어',
    'G-TELP(GLT)':'영어',
    'TOEFL-CBT':'영어',
    'TOEFL-IBT':'영어',
    'TOEFL-PBT':'영어',
    'SEPT':'영어',
    'TOEIC S&W':'영어',
    'GRE':'영어',
    'G-ETAT':'영어',
    'JLPT':'일본어',
    'JPT':'일본어',
    'JTRA':'일본어',
    'HSK':'중국어',
    '新HSK':'중국어', 
    '기타시험':'기타',
    'DELF':'기타',
    'DALF':'기타',
    'DELE':'기타',
    'TORFL':'기타'
})

In [12]:
X_train = df[:16570]
X_test = df[16570:]

In [13]:
numeric_features = ['근무경력','대학성적'] 
categorical_features = ['근무형태','출신대학','대학전공','어학시험','자격증']
binary_features = ['직종','세부직종','근무지역']
pca_features = ['직무태그']

X_train = X_train[numeric_features+categorical_features+binary_features+pca_features]  # 순서 주의!!!
X_test = X_test[numeric_features+categorical_features+binary_features+pca_features]

X_train.shape, X_test.shape

((16570, 11), (11048, 11))

In [14]:
def remove_outlier(X, q=0.05):  
    df = pd.DataFrame(X)
    return df.apply(lambda x: x.clip(x.quantile(q), x.quantile(1-q)), axis=0).values

numeric_transformer = Pipeline(
    steps=[
        ("imputer", SimpleImputer(strategy="mean")),
        ("outlier", FunctionTransformer(remove_outlier, kw_args={'q':0.05})), 
        ("scaler", PowerTransformer()),
    ]
)

categorical_transformer = Pipeline(
    steps=[
        ("imputer", SimpleImputer(strategy="most_frequent")), 
        ("encoder", OrdinalEncoder(handle_unknown='use_encoded_value', unknown_value=-1, dtype=int)),
    ]
)
    
binary_transformer = Pipeline(
    steps=[
        ("corpus", FunctionTransformer(lambda x: x.str.replace('·',',').str.split(',').str.join(" "))),
        ("BoW", CountVectorizer()),
    ]
)

pca_transformer = Pipeline(
    steps=[
        ("impuer", FunctionTransformer(lambda x: x.fillna('없음'))),      
        ("corpus", FunctionTransformer(lambda x: x.str.replace('·',',').str.split(',').str.join(" "))),
        ("BoW", CountVectorizer()),
        ("dense", FunctionTransformer(lambda x: x.todense(), accept_sparse=True)),
        ("pca", PCA(n_components=200)),
    ]
)

column_transformer = ColumnTransformer(
    transformers=[
        ("num", numeric_transformer, numeric_features),
        ("cat", categorical_transformer, categorical_features),
        ("bin1", binary_transformer, binary_features[0]), 
        ("bin2", binary_transformer, binary_features[1]), 
        ("bin3", binary_transformer, binary_features[2]), 
        ("pca1", pca_transformer, pca_features[0]), 
   ]
)

preprocessor = Pipeline(
    steps=[
        ("column", column_transformer), 
        ("selector", SelectPercentile(percentile=80)),
    ]
)

model = Pipeline(
    steps=[
        ("preprocessor", preprocessor), 
        ("classifier", LGBMRegressor(random_state=0)),
    ]
)

set_config(display="diagram")  # To view the text pipeline, change to display='text'.
model

Pipeline(steps=[('preprocessor',
                 Pipeline(steps=[('column',
                                  ColumnTransformer(transformers=[('num',
                                                                   Pipeline(steps=[('imputer',
                                                                                    SimpleImputer()),
                                                                                   ('outlier',
                                                                                    FunctionTransformer(func=<function remove_outlier at 0x000001FD25A31550>,
                                                                                                        kw_args={'q': 0.05})),
                                                                                   ('scaler',
                                                                                    PowerTransformer())]),
                                                                   ['근무경력',
                                                                    '대학성적']),
                                                                  ('cat',
                                                                   Pipeline(steps=[('imputer',
                                                                                    SimpleImputer(strategy='...
                                                                                    FunctionTransformer(func=<function <lambda> at 0x000001FD25A31670>)),
                                                                                   ('corpus',
                                                                                    FunctionTransformer(func=<function <lambda> at 0x000001FD25E801F0>)),
                                                                                   ('BoW',
                                                                                    CountVectorizer()),
                                                                                   ('dense',
                                                                                    FunctionTransformer(accept_sparse=True,
                                                                                                        func=<function <lambda> at 0x000001FD25E80280>)),
                                                                                   ('pca',
                                                                                    PCA(n_components=200))]),
                                                                   '직무태그')])),
                                 ('selector',
                                  SelectPercentile(percentile=80))])),
                ('classifier', LGBMRegressor(random_state=0))])

In [15]:
sscv = ShuffleSplit(test_size=.3334, n_splits=5, random_state=0) # Public LB 사이즈와 동일하게 평가데이터 사이즈 설정 
scores = cross_val_score(model, X_train, y_train, scoring='neg_mean_squared_error', cv=sscv)

print("Default LGBM CV scores: ", np.sqrt(-1*scores))
print("Default LGBM CV mean = %.2f" % np.sqrt(-1*scores.mean()), "with std = %.2f" % np.sqrt(scores.std()))

Default LGBM CV scores:  [851.39993891 824.83593771 862.31372433 875.95211967 863.40232987]
Default LGBM CV mean = 855.75 with std = 171.03


In [16]:
%%time

param_distributions = {
    "preprocessor__column__num__imputer__strategy": CategoricalDistribution(["mean"]),
    "preprocessor__column__num__outlier__kw_args": CategoricalDistribution([{'q':0.02},{'q':0.025}]),
    "preprocessor__column__pca1__pca__n_components": IntDistribution(200,400,step=100),  
    "preprocessor__selector__percentile": IntDistribution(70,100,step=10),  
}

optuna_search = OptunaSearchCV(model, 
                               param_distributions, 
                               cv=sscv, scoring='neg_mean_squared_error', 
                               n_trials=20,
                               timeout=60, 
                               study=optuna.create_study(sampler=optuna.samplers.TPESampler(seed=100), direction="maximize"))

optuna.logging.set_verbosity(optuna.logging.WARNING)
optuna_search.fit(X_train, y_train)

print(f"Best params: {optuna_search.best_params_}")
print("Best score: %.2f" % (-1*optuna_search.best_score_)**0.5)

[I 2022-11-25 00:25:13,367] A new study created in memory with name: no-name-b6636ae6-4e56-4410-9c34-f786c76ba1ae


Best params: {'preprocessor__column__num__imputer__strategy': 'mean', 'preprocessor__column__num__outlier__kw_args': {'q': 0.02}, 'preprocessor__column__pca1__pca__n_components': 300, 'preprocessor__selector__percentile': 100}
Best score: 842.99
CPU times: total: 9min 39s
Wall time: 1min 32s


In [17]:
# 최적값으로 파이프라인 재설정
model.set_params(**optuna_search.best_params_)

# 전처리 파이프라인만 수행
X_train = preprocessor.fit_transform(X_train, y_train)
X_test = preprocessor.transform(X_test)

In [18]:
tuner = LightGBMTunerCV(
    params={
        "objective": "regression",   # 지도학습 유형(regression/binary/multiclass)
        "metric": "rmse",
        "verbosity": -1,             # 진행과정 출력안함
        "boosting_type": "gbdt",     # 실행하고자 하는 알고리즘 유형(gdbt/rf/dart/goss)
        "seed": 0,
        "boost_from_average" : False,
        "Approximate method" : "exact",
        "num_iterations": 1000,
        #"learning_rate": 0.05,
        "max_depth": 30,
        #"feature_fraction": 0.9
    },
    train_set=lgb.Dataset(X_train, y_train), # LGBM 데이터셋으로 변환
    nfold=5,
    num_boost_round=500,                     # boosting 반복횟수
    callbacks=[lgb.early_stopping(100)],     # 모든 정지 라운드 동안 validation 성능이 개선되지 않으면 조기종료
    time_budget=120,                          # 튜닝 허용 최대시간(초단위)
    optuna_seed=0,
    
)

tuner.run()

feature_fraction, val_score: inf:   0%|                                                          | 0/7 [00:00<?, ?it/s]

Training until validation scores don't improve for 100 rounds


feature_fraction, val_score: 833.703948:  14%|######1                                    | 1/7 [00:06<00:36,  6.12s/it]

Early stopping, best iteration is:
[223]	cv_agg's rmse: 833.704 + 3.68446


feature_fraction, val_score: 833.703948:  29%|############2                              | 2/7 [00:12<00:32,  6.45s/it]

...

num_leaves, val_score: 833.453593:   5%|##4                                             | 1/20 [00:46<14:40, 46.33s/it]

Early stopping, best iteration is:
[247]	cv_agg's rmse: 833.454 + 2.74204


num_leaves, val_score: 833.453593:  10%|####8                                           | 2/20 [02:03<18:34, 61.93s/it]


Early stopping, best iteration is:
[247]	cv_agg's rmse: 833.454 + 2.74204


  0%|                                                                                            | 0/5 [00:00<?, ?it/s]


In [19]:
print(f"\nBest params: {tuner.best_params}")
print(f"\nBest score: {tuner.best_score:.2f}")


Best params: {'objective': 'regression', 'metric': 'rmse', 'verbosity': -1, 'boosting_type': 'gbdt', 'seed': 0, 'boost_from_average': False, 'Approximate method': 'exact', 'num_iterations': 1000, 'feature_pre_filter': False, 'lambda_l1': 0.0, 'lambda_l2': 0.0, 'num_leaves': 31, 'feature_fraction': 0.8999999999999999, 'bagging_fraction': 1.0, 'bagging_freq': 0, 'min_child_samples': 20}

Best score: 833.45


In [20]:
# 최적화된 하이퍼파라미터로 OOF를 수행하여 최종 LGBM 모형 생성
models = cross_validate(LGBMRegressor(**tuner.best_params), # 최적화된 hyperparameter 사용
                        X_train, y_train, 
                        cv=sscv, 
                        scoring='neg_mean_squared_error', 
                        return_estimator=True)
oof_pred = np.array([m.predict(X_test) for m in models['estimator']]).mean(axis=0)

scores = models['test_score']
print("\nTuned LGBM CV scores: ", np.sqrt(-1*scores))
print("Tuned LGBM CV mean = %.2f" % np.sqrt(-1*scores.mean()), "with std = %.2f" % np.sqrt(scores.std()))

[LightGBM] [Warning] Unknown parameter: Approximate
[LightGBM] [Warning] Unknown parameter: method
[LightGBM] [Warning] feature_fraction is set=0.8999999999999999, colsample_bytree=1.0 will be ignored. Current value: feature_fraction=0.8999999999999999
[LightGBM] [Warning] lambda_l1 is set=0.0, reg_alpha=0.0 will be ignored. Current value: lambda_l1=0.0
[LightGBM] [Warning] bagging_fraction is set=1.0, subsample=1.0 will be ignored. Current value: bagging_fraction=1.0
[LightGBM] [Warning] lambda_l2 is set=0.0, reg_lambda=0.0 will be ignored. Current value: lambda_l2=0.0
[LightGBM] [Warning] bagging_freq is set=0, subsample_freq=0 will be ignored. Current value: bagging_freq=0
[LightGBM] [Warning] Unknown parameter: Approximate
[LightGBM] [Warning] Unknown parameter: method
[LightGBM] [Warning] feature_fraction is set=0.8999999999999999, colsample_bytree=1.0 will be ignored. Current value: feature_fraction=0.8999999999999999
[LightGBM] [Warning] lambda_l1 is set=0.0, reg_alpha=0.0 will 

In [21]:
# submission 화일 생성
filename = f'lgbm_{LGBM_VERSION}_{np.sqrt(-1*scores.mean()):.2f}.csv'
pd.DataFrame({'ID':test_id, 'Salary':oof_pred}).to_csv(filename, index=False)